In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import subprocess

# Open the file containing the tRNA scan results
trna_=open("combined_trna_scan_v2.txt")
trna=trna_.readlines()
trna_.close()

# Open the file containing the ribozyme scan results
rnarobo_=open("rnarobo_result_dereplicated.txt")
rnarobo=rnarobo_.readlines()
rnarobo_.close()

# Create a dictionary to store the starting positions of each tRNA
dicti={}
# Loop through each line of the tRNA scan results, skipping the header
for line in trna[3:]:
    # Split the line into a list of values
    line=line.strip().split()
    # If the sequence name is already in the dictionary, append the starting position to the existing list
    if line[0] in dicti:
        dicti[line[0]].append(int(line[3]))
    # If the sequence name is not in the dictionary, create a new list with the starting position
    else:
        dicti[line[0]]=[int(line[3])]

# Create a dictionary to store the starting positions of each ribozyme
dicti_r={}
# Loop through each line of the ribozyme scan results, skipping the header and footer
for line in rnarobo[11:-6]:
    # If the line does not start with "|", it contains ribozyme information
    if line[0]!="|":
        # Replace any periods in the line with underscores
        line=line.replace(".","_")
        # Split the line into a list of values
        line=line.strip().split()
        # If the scaffold ID is already in the dictionary, append the starting position to the existing list
        if line[2] in dicti_r:
            dicti_r[line[2]].append(int(line[0]))
        # If the scaffold ID is not in the dictionary, create a new list with the starting position
        else:
            dicti_r[line[2]]=[int(line[0])]

# Create an empty list to store the matching tRNA and ribozyme pairs
matching=[]
# Loop through each scaffold ID in the ribozyme dictionary
for key in dicti_r:
    # If the scaffold ID is also in the tRNA dictionary
    if key in dicti:
        # Loop through each ribozyme starting position for the current scaffold ID
        for value in dicti_r[key]:
            # Loop through each tRNA starting position for the current sequence name
            for value2 in dicti[key]:
                # If the difference between the ribozyme and tRNA starting positions is within 5 nucleotides
                if value-value2 in range(-5,6):
                    # Append the scaffold ID, ribozyme starting position, and tRNA starting position to the matching list
                    matching.append([key,value,value2])

# Display the matching list
matching

In [ ]:
import pandas as pd
import numpy as np

# Create an empty DataFrame with the specified column names
lets_do_this=pd.DataFrame(columns=["ID","trna_nr","trna_start","trna_end","trna_type","codon","ins_start","ins_end","trna_confidence","psuedo","rnarobo_start","rnarobo_end","rnarobo_seq"])

# Initialize a counter and an empty list for duplicate entries
check=0
duplicates=[]

# Loop through each matching ribozyme and tRNA pair
for nr in range(len(newm)):
    match=newm[nr]
    lin_df=""
    
    # Loop through each line of the tRNA scan results, skipping the header
    for line in trna[3:]:
        # Split the line into a list of values
        line=line.strip().split()
        
        # If the sequence name matches the current ribozyme and the starting position matches the current tRNA
        if match[0]==line[0]:
            if len(lin_df.split())==0:
                if str(match[2])==line[3]:
                    lin_df+=" ".join(line)+" "
                    if lin_df[-2]!="o":
                        lin_df+="NA "
            else: 
                duplicates.append(line)
    
    # Loop through each line of the ribozyme scan results, skipping the header and footer
    for line in rnarobo[11:-6]:
        # Replace any periods in the line with underscores
        line=line.replace(".","_")
        # Split the line into a list of values
        line=line.strip().split()
        
        # If the current line contains ribozyme information
        if len(line)>1:
            # If the scaffold ID matches the current ribozyme and the starting position matches the current ribozyme
            if match[0]==line[2]:
                if str(match[1])==line[0]:
                    lin_df+=" ".join(line[:2])
                    lin_df+=" "
                    check=1
    
    # Split the final string into a list of values and add it as a new row to the DataFrame
    lin_f=lin_df.split()
    lets_do_this.loc[nr]=np.array(lin_f)

# Save the intermediate results to a file and reload it as a pandas DataFrame
df1str=lets_do_this.to_csv(sep="\t")
f= open("comb_trna_v2_plus_ribo.csv","w+")
f.write(df1str)
f.close()
fff=pd.read_csv("comb_trna_v2_plus_ribo.csv",sep="\t",index_col=0)

In [ ]:
import subprocess
import pandas as pd

# Add two new columns to the DataFrame and initialize them with empty strings
fff["trnaseq"]= " "
fff["ribozyme_control"]= " "

# Loop through each row in the DataFrame
for ID in range(len(fff)):
    # Select the correct contig/genome from the rnarobo hit
    inp="grep -A1 -m1 '"+fff.iloc[ID,0]+"' combined_all_viral_sequences_v2.fasta" 
    res=subprocess.run(inp,shell=True, capture_output=True, text=True)
    sequ=res.stdout.split()[-1]
    start=fff.iloc[ID,2]-1
    end=fff.iloc[ID,3]
    start2=fff.iloc[ID,10]-1
    end2=fff.iloc[ID,11]
    
    # If the start position is less than the end position
    if start < end:
        # Extract the tRNA and ribozyme sequences
        trna=sequ[start:end]
        rnarobo=sequ[start2:end2]
        fff.iloc[ID,-2]=trna
        fff.iloc[ID,-1]=rnarobo
    # If the end position is less than the start position
    if end < start:
        # Reverse the sequence and extract the tRNA and ribozyme sequences
        end-=1
        start+=1
        end2-=1
        start2+=1
        trna_rev=sequ[end:start]
        ribo_rev=sequ[end2:start2]
        trna=[]
        rnarobo=[]
        # Transcribe the reversed tRNA and ribozyme sequences back into the correct format
        for nuc in range(len(trna_rev)-1,-1,-1):
            if trna_rev[nuc]=="A":
                trna.append("T")
            if trna_rev[nuc]=="T":
                trna.append("A")
            if trna_rev[nuc]=="G":
                trna.append("C")
            if trna_rev[nuc]=="C":
                trna.append("G")
            if trna_rev[nuc]=="N":
                trna.append("N")
        for nuc in range(len(ribo_rev)-1,-1,-1):
            if ribo_rev[nuc]=="A":
                rnarobo.append("T")
            if ribo_rev[nuc]=="T":
                rnarobo.append("A")
            if ribo_rev[nuc]=="G":
                rnarobo.append("C")
            if ribo_rev[nuc]=="C":
                rnarobo.append("G")
            if ribo_rev[nuc]=="N":
                rnarobo.append("N")
        trna="".join(trna)
        rnarobo="".join(rnarobo)
        fff.iloc[ID,-2]=trna
        fff.iloc[ID,-1]=rnarobo

# Check if the control and actual sequences are the same
for x in range(len(fff)):
    seq1=fff.loc[x,"rnarobo_seq"]
    seq3="".join(seq1.split("|"))
    seq2=fff.loc[x,"ribozyme_control"]
    if seq2!=seq3:
        print("something wrong",x)
        print(seq2)
        print(seq3)

In [ ]:
f=fff.copy(deep=True)
#create new columns for study origin
f.loc[:,"study_origin"]=""

#manually add the study origin

f.loc[0:59,"study_origin"]="1"
f.loc[59:299,"study_origin"]="2"
#three: not found
f.loc[299:315,"study_origin"]="4"
f.loc[315:679,"study_origin"]="5"
f.loc[679:680,"study_origin"]="6"
#7 not present
f.loc[681:,"study_origin"]="8"
f["study_origin"].value_counts()

In [ ]:
print ("Unique ribos: "+str(len(f["ribozyme_control"].unique())))
print ("Unique trnas: "+str(len(f["trnaseq"].unique())))
#combinme ribo+trnaseq
f["comb_seq"]=f["ribozyme_control"]+f["trnaseq"]
print ("Unique ribo/trna_combinations: "+str(len(f["comb_seq"].unique())))

Unique ribos: 126


In [159]:
df1str=f.to_csv(sep="\t",index=False)
f= open("comb_trna_seq_plus_ribo_fix_march.csv","w+")
f.write(df1str)
f.close()